## PREPARACION DE LOS DATOS

In [1]:
# Importacion de Numpy y Pandas
import numpy as np
import pandas as pd

# Importacion de PyETo
import pyeto

# Importacion de Datetime
import datetime

# Importacion de monthrange
from calendar import monthrange

# Deshabilitamos mensajes de warning
pd.options.mode.chained_assignment = None

In [2]:
def rad_solar_from_temp(year, month, day, latitude, longitude, altitude, coastal, temp_min, temp_max):
    """
    Parametros
    ----------
    Entrada
        year : int
            Año a evaluar
        month,  : int
            Mes a evaluar
        day : int
            Dia a evaluar
        latitude : float
            Latitud de base meteorologica (grados decimales)
        longitude : float
            Longitud de base meteorologica (grados decimales)
        altitude : list
            Altitud sobre el nivel del mar (metros)
        coastal : boolean
            True  -> zona costera
            False -> zona fuera de la costa
        temp_min : float
            Temperatura minima (grados centigrados)
        temp_max : float
            Temperatura maxima (grados centigrados)
    ----------
    Salida
        rad_sol_from_t : float
            Radiacion Solar (MJ m-2 day-1) 
    """
    # Calculo del dia contiguo en referencia al año completo
    day_of_year = datetime.date(year,month,day).timetuple().tm_yday
    
    # Conversion latitud de grados decimales a radianes
    latitude_rad = pyeto.deg2rad(latitude)

    # Conversion longitud de grados decimales a radianes
    longitude_rad = pyeto.deg2rad(longitude)

    # Calculo declinacion solar para el dia del año a evaluar
    sol_dec = pyeto.sol_dec(day_of_year)

    # Calculo angulo al atardecer (Ws) respecto a latitud y declinacion solar
    sha = pyeto.sunset_hour_angle(latitude_rad, sol_dec)

    # Calculo relacion inversa de distancia entre tierra y sol para dia del año a evaluar
    ird = pyeto.inv_rel_dist_earth_sun(day_of_year)

    # Estimacion de radiacion extraterrestrial (Ra, a nivel radiacion atmposferica) 
    et_rad = pyeto.et_rad(latitude_rad, sol_dec, sha, ird)

    # Estimacion radiacion cielo abierto respecto a altitud y radiacion extraterrestrial
    cs_rad = pyeto.cs_rad(altitude, et_rad)
    
    # Estimacion de radiacion soloar (u onda corta), Rs, basado en temp minima y maxima
    rad_sol_from_t = pyeto.sol_rad_from_t(et_rad, cs_rad, temp_min, temp_max, coastal)
    
    # Retornamos rad_sol_from_t en MJm-2day-1
    return rad_sol_from_t

In [3]:
def rad_solar_from_sun_hours(year, month, day, latitude, longitude, altitude, sun_hours):
    """
    Parametros
    ----------
    Entrada
        year : int
            Año a evaluar
        month,  : int
            Mes a evaluar
        day : int
            Dia a evaluar
        latitude : float
            Latitud de base meteorologica (grados decimales)
        longitude : float
            Longitud de base meteorologica (grados decimales)
        altitude : list
            Altitud sobre el nivel del mar (metros)
        coastal : boolean
            True  -> zona costera
            False -> zona fuera de la costa
        sun_hours : float
            Horas sul durante el dia (horas)
    ----------
    Salida
        rad_sol_from_sun_hours : float
            Radiacion Solar (MJ m-2 day-1) 
    """
    # Calculo del dia contiguo en referencia al año completo
    day_of_year = datetime.date(year,month,day).timetuple().tm_yday
    
    # Conversion latitud de grados decimales a radianes
    latitude_rad = pyeto.deg2rad(latitude)

    # Conversion longitud de grados decimales a radianes
    longitude_rad = pyeto.deg2rad(longitude)

    # Calculo declinacion solar para el dia del año a evaluar
    sol_dec = pyeto.sol_dec(day_of_year)

    # Calculo angulo al atardecer (Ws) respecto a latitud y declinacion solar
    sha = pyeto.sunset_hour_angle(latitude_rad, sol_dec)

    # Calculo relacion inversa de distancia entre tierra y sol para dia del año a evaluar
    ird = pyeto.inv_rel_dist_earth_sun(day_of_year)

    # Estimacion de radiacion extraterrestrial (Ra, a nivel radiacion atmposferica) 
    et_rad = pyeto.et_rad(latitude_rad, sol_dec, sha, ird)

    # Estimacion radiacion cielo abierto respecto a altitud y radiacion extraterrestrial
    cs_rad = pyeto.cs_rad(altitude, et_rad)
    

    # Calculate incoming solar (or shortwave) radiation, 
    # Rs (radiation hitting a horizontal plane after scattering by the atmosphere) 
    # from relative sunshine duration.
    daylight_hours = pyeto.daylight_hours(sha)
    #print("daylight_hours = %s" % daylight_hours)

    # Estimacion de radiacion soloar (u onda corta), Rs, basado en horas sol
    rad_sol_from_sun_hours = pyeto.sol_rad_from_sun_hours(daylight_hours, sun_hours, et_rad)
    
    # Retornamos rad_sol_from_sun_hours en MJm-2day-1
    return rad_sol_from_sun_hours

In [4]:
# Funcion para generar los archivos correspondientes a los meses especificados como requerimiento para calculo ET0
def data_req_for_et0(conagua_year_data, month_list, year, latitude, longitude, altitude, coastal, export_file):
    """
    Parametros
    ----------
    Entrada
        conagua_year_data : str
            Ruta del archivo anualizado Conagua 
        month_list,  : list
            Lista de los meses a preparar
        year : int
            Año de referencia
        latitude : float
            Latitud de base meteorologica (grados decimales)
        longitude : float
            Longitud de base meteorologica (grados decimales)
        altitude : list
            Altitud sobre el nivel del mar (meotrs)
        coastal : boolean
            True  -> zona costera
            False -> zona fuera de la costa
        export_file : boolean
            True  -> guardar archivo en ../output/preparation/
            False -> se despliega en consola los datos preparados
    ----------
    Salida
        return : int
            0
    """
    ###########################################################################################################
    #display("data_req_for_et0 %s-%s" %(month,year))
    # Asinamos la ruta del archivo con datos anualizados Conagua a una variable
    xls = pd.ExcelFile(conagua_year_data)

    # Ciclo for para iterar sobre la lista de meses
    for month in month_list:
        # Creación del dataframe a partir del archivo excel cargado usando el tab del mes
        conagua_df = pd.read_excel(xls, month)
        
        # Renombrado de encabezado de columnas de mayusculas a minusculas
        conagua_df.columns = [col.lower() for col in conagua_df]

        # Renombrado de encabezados de columnas principales para eliminar sus acronimos
        conagua_df.rename(columns={
                                    't.s': 'termometro seco', 
                                    't.h': 'termometro humedo',
                                    't.v': 'tension de vapor',
                                    'max': 'temperatura maxima',
                                    'min': 'temperatura minima',
                                    'p.r': 'punto de rocio',
                                    'h.r': 'humedad relativa',
                                    'max.1': 'humedad maxima',
                                    'min.1': 'humedad minima',
                                    'psen':'presion de la estacion',
                                    'max.2': 'presion de la estacion maxima',
                                    'min.2': 'presion de la estacion minima',
                                    'prnm':'presion residual al nivel del mar',
                                    'max.3': 'presion residual al nivel del mar maxima',
                                    'min.3': 'presion residual al nivel del mar minima',
                                    'max.4': 'evaporacion maxima',
                                    'min.4': 'evaporacion minima',
                                    'direcc': 'direccion dominante',
                                    'vel' : 'velocidad dominante',
                                    'direcc.1' : 'direcciom maxima',
                                    'vel.1' : 'velocidad maxima',
                                    'media' : 'velocidad media',
                                    'p.1': 'pluviometro'
        }, inplace=True)
        #display("Paso >> Cargado de archivo anualizado y renombramiento de columnas")
        #display("Paso >> Cargado de archivo anualizado y renombramiento de columnas >> conagua_df")
        #display(conagua_df)
        #display("Paso >> Cargado de archivo anualizado y renombramiento de columnas >> conagua_df.info()")
        #display(conagua_df.info())
        #display("Paso >> Cargado de archivo anualizado y renombramiento de columnas >> conagua_df.isnull().sum()")
        #display(conagua_df.isnull().sum())
        ###########################################################################################################
        # Definicion de lista que contiene las columnas dentro de conagua_df
        columns_conagua_df = list(conagua_df.columns)
        
        #Definimos la lista que contiene el nombre de las columnas preservar dentro de conagua_df
        columns_remaining = [
                                    'dia',
                                    'tension de vapor',
                                    'temperatura maxima',
                                    'temperatura minima',
                                    'humedad relativa',
                                    'humedad minima',
                                    'humedad maxima',
                                    'inten',
                                    'velocidad dominante',
                                    'presion de la estacion',
                                    'pluviometro'
        ]

        # A traves de la funcion set realizamos la substraccion
        # Cada elemento de columns_conagua_df se buscara dentro de columns_remaining
        # Si no se encuentra coincidencia alguna esta se guarda como elemento de columns_to_drop
        columns_to_drop_set = set(columns_conagua_df)-set(columns_remaining)
        columns_to_drop = sorted(columns_to_drop_set, key = lambda k : columns_conagua_df.index(k))

        # Removemos columns_to_drop de conagua_df y guardamos en conagua_df
        conagua_df.drop(columns=columns_to_drop,inplace=True)
        
        # https://eto.readthedocs.io/en/latest/package_references.html
        # Renombramiento de columnas necesarias para calculacion de ETo
        conagua_df.rename(columns={
                                    'tension de vapor': 'e_a',
                                    'temperatura maxima': 'T_max', 
                                    'temperatura minima': 'T_min',
                                    'humedad relativa': 'RH_mean',
                                    'humedad minima': 'RH_min',
                                    'humedad maxima': 'RH_max',
                                    'inten': 'n_sun',
                                    'velocidad dominante': 'U_z',
                                    'presion de la estacion': 'P',
                                    'pluviometro': 'rain_mm'
        }, inplace=True)

        #display(conagua_df)
        #display("Paso -> Eliminación de columnas no requeridas")
        #display("Paso >> Eliminación de columnas no requeridas >> conagua_df")
        #display(conagua_df)
        #display("Paso >> Eliminación de columnas no requeridas >> conagua_df.info()")
        #display(conagua_df.info())
        #display("Paso >> Eliminación de columnas no requeridas >> conagua_df.isnull().sum()")
        #display(conagua_df.isnull().sum())
        ###########################################################################################################
        # Generamos una nueva columna la cual tiene como proposito hacer una copia directa
        # de la columna dia a dia_index para ser usada como indice del dataframe
        conagua_df['dia_index'] = ''
        conagua_df['dia_index'] = conagua_df['dia']

        # Definimos el index en conagua_df para la columna dia_index
        conagua_df.set_index('dia_index', inplace = True)

        # Calcular dias disponibles en el mes para acotar los contenidos extras a la tabla de referencia Conagua
        # Obtencion de numeros de dias en el mes a evaluar
        monthrange_result = monthrange(year,month_list.index(month)+1) 
        weekday_of_first_day_of_month = monthrange_result[0]
        number_of_days_in_month = monthrange_result[1]

        # Recortamos nuestro daframe al contexto de dias dentro del mes a evaluar
        conagua_df = conagua_df.iloc[1:number_of_days_in_month+1]

        # Remplazamor cualquier 0 tipo int/string por Nan 
        conagua_df[conagua_df.columns] = conagua_df[conagua_df.columns].replace({'0':np.nan, 0:np.nan})

        # Obtenemos informacion del total de valores nulos presentes en conagua_df
        conagua_isnull = conagua_df.isnull().sum()

        # Lista de columnas con valores NaN presentes 
        conagua_columns_isnull = list(conagua_isnull.index.values.tolist())

        # Definicion de lista vacia con informacion de las columnas a remover que tengan un valor NaN
        conagua_columns_nan = []

        # Checamos que columnas cuentan con valores nulos y lo agregamos a conagua_columns_dropna
        for i in range(0,len(conagua_isnull)):
            if conagua_isnull[conagua_columns_isnull[i]] > 0:
                conagua_columns_nan.append(conagua_columns_isnull[i])
        
        # Realizamos la imputacion de aquellos valores NaN en la columna tomando el valor anterior
        for each in conagua_columns_nan:
            # Para el caso de la columna lluvia, buscamos dejar en 0 aquellos campos vacios
            if(each == 'rain_mm'):
                conagua_df[each].fillna(0, inplace = True)
            else:
                conagua_df[each].fillna(method = 'ffill', inplace = True)
                conagua_df[each].fillna(method = 'bfill', inplace = True) 

        
        #display("Paso -> Conversión de valores cero a NaN e imputación de NaN con datos de día adyacentes")
        #display("Paso >> Conversión de valores cero a NaN e imputación de NaN con datos de día adyacentes >> conagua_df")
        #display(conagua_df)
        #display("Paso >> Conversión de valores cero a NaN e imputación de NaN con datos de día adyacentes >> conagua_df.info()")
        #display(conagua_df.info())
        #display("Paso >> Conversión de valores cero a NaN e imputación de NaN con datos de día adyacentes >> conagua_df.isnull().sum()")
        #display(conagua_df.isnull().sum())
        ###########################################################################################################
        # Creacion de la columna fecha en conagua_df
        conagua_df['date'] = None
        conagua_df['dia'] = conagua_df['dia'].astype(int)
        
        # Iteramos en cada uno de los valores de la columna dia para construir el valor correspondiente a su fecha 
        # partir de concatenar año/mes/dia en la columna previamente insertada en conagua_df
        for i in range(conagua_df['dia'][1],len(conagua_df['dia'])+1):
            conagua_df['date'][i] = str(year)+'-'+str(month_list.index(month)+1)+'-'+str(conagua_df['dia'][i])

        #Conversión de la columna "date" al formato de serie de tiempo
        conagua_df['date'] = pd.to_datetime(conagua_df['date'])

        #display("Paso -> Creación de columna fecha y conversión a formato de tiempo")
        #display("Paso >> Creación de columna fecha y conversión a formato de tiempo >> conagua_df")
        #display(conagua_df)
        #display("Paso >> Creación de columna fecha y conversión a formato de tiempo >> conagua_df.info()")
        #display(conagua_df.info())
        #display("Paso >> Creación de columna fecha y conversión a formato de tiempo >> conagua_df.isnull().sum()")
        #display(conagua_df.isnull().sum())
        ###########################################################################################################
        # Hacemos una inspeccion para la seccion de temperaturas diarias
        # garantizando que la T_max sea mayor que la T_min
        # En caso de no cumplirse se invertiran los valores
        # y evitar problemas en el apartado del calculo de radiacion solar. 
        for day in conagua_df['dia']:
            ##display(day,conagua_df['T_min'][day],conagua_df['T_max'][day])
            if (conagua_df['T_max'][day] < conagua_df['T_min'][day]):
                temp_t_max = conagua_df['T_max'][day]
                temp_t_min = conagua_df['T_min'][day]
                conagua_df['T_max'][day] = temp_t_min
                conagua_df['T_min'][day] = temp_t_max
        
        #display("Paso -> Inspección de temperatura máxima Vs temperatura mínima")
        #display("Paso >> Inspección de temperatura máxima Vs temperatura mínima >> conagua_df")
        #display(conagua_df)
        #display("Paso >> Inspección de temperatura máxima Vs temperatura mínima >> conagua_df.info()")
        #display(conagua_df.info())
        #display("Paso >> Inspección de temperatura máxima Vs temperatura mínima >> conagua_df.isnull().sum()")
        #display(conagua_df.isnull().sum())
        ###########################################################################################################        
        # Agregando columna para depositar valores calculados de Radiacion Solar a partir de Temperaturas
        conagua_df['R_s'] = ''        

        # Anadimos para cada registro de la tabla su radiacion solar a partir de temperaturas
        for day in conagua_df['dia']: 
            conagua_df['R_s'][day] = rad_solar_from_sun_hours(year,month_list.index(month)+1,day,latitude,longitude,altitude,conagua_df['n_sun'][day])
            #conagua_df['R_s'][day] = rad_solar_from_temp(year,month_list.index(month)+1,day,latitude,longitude,altitude,coastal,conagua_df['T_min'][day],conagua_df['T_max'][day])
            
        # Arreglamos cualquier error producido por datos tipo flotante en columna de radiacion solar
        conagua_df['R_s'] = conagua_df['R_s'].astype(float, errors = 'raise')

        # Conversion de Hecto Pascales a Kilo Pascales en tension de vapor
        #1 hectopascal [hPa] = 0.1 kilopascal [kPa]
        conagua_df['e_a'] = conagua_df['e_a']*0.1

        # Conversion de Hecto Pascales a Kilo Pascales en presion de la estacion
        conagua_df['P'] = conagua_df['P']*0.1

        #display("Paso -> Creación de columna y cálculo de radiación solar")
        #display("Paso >> Creación de columna y cálculo de radiación solar >> conagua_df")
        #display(conagua_df)
        #display("Paso >> Creación de columna y cálculo de radiación solar >> conagua_df.info()")
        #display(conagua_df.info())
        #display("Paso >> Creación de columna y cálculo de radiación solar >> conagua_df.isnull().sum()")
        #display(conagua_df.isnull().sum())
        ###########################################################################################################
        # Eliminacion de la columna dia
        conagua_df.drop(columns=['dia'],inplace=True)

        # Seteando el index para la columna dia
        conagua_df.set_index('date', inplace = True)

        # Redondeamos todos los valores de los parametros a 2 decimales
        conagua_df[conagua_df.columns] = conagua_df[conagua_df.columns].round(2)

        # Guardado del Archivo en formato CSV
        if(export_file == True):
            conagua_df.to_csv('../output/preparation/conagua_' + str(year) + '_' + month.lower() + '_req_for_eto.csv')
        # Si no se requiere guardado, se visualiza el reordenamiento de datos para ETo
        elif(export_file == False):
            display("Datos Conagua %s-%s preparados para ETo" %(month,year))
            #display("Paso >> Guardado/Visualización datos preparados ETo >> conagua_df")
            display(conagua_df)
            #display("Paso >> Guardado/Visualización datos preparados ETo >> conagua_df.info()")
            #display(conagua_df.info())
            #display("Paso >> Guardado/Visualización datos preparados ETo >> conagua_df.isnull().sum()")
            #display(conagua_df.isnull().sum())
        ###########################################################################################################
    return 0

In [5]:
# Definicion de anio para busqueda de archivo de conagua con datos 
year = 2020

# Meses a preparar en resumen de datos
#month_list=['ENERO','FEBRERO','MARZO','ABRIL','MAYO','JUNIO','JULIO','AGOSTO','SEPTIEMBRE','OCTUBRE','NOVIEMBRE','DICIEMBRE']
month_list=['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']

# Definicion de latitud, longitude, altitude, coastal para la seccion del calculo de R_s 
latitude = 19.59216667
longitude = -103.59075
altitude = 1408.99
coastal = False

# Guardar archivo
export_file = True

# Localizacion del archivo de Conagua que contempla los datos climatologicos de un año completo
conagua_year_data = '../data/conagua/clima_'+str(year)+'.xlsx'
# Ruta de archivo en sistema operativo windows
# conagua_year_data = "C:/Users/aabaltaz/OneDrive - Intel Corporation/Documents/programacion_analisis_datos/requerimiento_agua_tomate_rojo/datos/conagua/clima_"+str(year)+".xlsx"

# Llamar a funcion para preparar los datos de cada mes
data_req_for_et0(conagua_year_data, month_list, year, latitude, longitude, altitude, coastal, export_file)


0